In [1]:
import _dataset_tools as dt
import _my_tools as mt
import cv2 as cv
import numpy as np
import os

### Image cropping
All of the grayscale radar images from folder `datasets/dataset_full/` are taken and squares of size $96\times96$ are cropped from them with a stride of size $48$. The cropped squares are saved in `datasets/cropped_96_96/` in folder according to the square's coordinates.

In [ ]:
out_f = "datasets/cropped_96_96/"
in_f = "datasets/dataset_full/"

for f in range(55):
    os.mkdir(out_f+str(f).zfill(2))

In [ ]:
dt.cropFolder(in_f,out_f,96,48)

### Cleaning and Triplets finding
All images from every folder of `datasets/cropped_96_96/` are taken, and images that have more than $95\%$ of the area without precipitation or have only precipitation of level $1$ (of $16$) are deleted. The previous images may also be removed so that there are in the end always left three consecutive images.

In [ ]:
in_f = "datasets/cropped_96_96/"
for folder in sorted(os.listdir(in_f)):
    dt.findTriplets(in_f+folder+"/")

### Loading to NumPy array

In [2]:
in_f = "datasets/cropped_96_96/"
folders = sorted(os.listdir(in_f))
# -------------------------------------------------------
X,y = dt.loadToNPA(in_f + folders[0] + "/")
print(X.shape, " : ", y.shape)
# -------------------------------------------------------
for i in range(1,55):
    print("Processing folder ", in_f, folders[i])
    
    X_tmp,y_tmp = dt.loadToNPA(in_f+folders[i]+"/")
    X_tmp = np.rot90(X_tmp,k=i%4,axes=(2,3))
    y_tmp = np.rot90(y_tmp,k=i%4,axes=(1,2))
    
#     mt.showImg("",X_tmp[0,0],y_tmp[0],X_tmp[0,1])
    
    X = np.concatenate((X,X_tmp),axis=0)
    y = np.concatenate((y,y_tmp),axis=0)
    
    print(X.shape, " : ", y.shape)

(2947, 2, 96, 96)  :  (2947, 96, 96)
Processing folder  datasets/cropped_96_96/ 01
(6330, 2, 96, 96)  :  (6330, 96, 96)
Processing folder  datasets/cropped_96_96/ 02
(9846, 2, 96, 96)  :  (9846, 96, 96)
Processing folder  datasets/cropped_96_96/ 03
(13259, 2, 96, 96)  :  (13259, 96, 96)
Processing folder  datasets/cropped_96_96/ 04
(16664, 2, 96, 96)  :  (16664, 96, 96)
Processing folder  datasets/cropped_96_96/ 05
(20039, 2, 96, 96)  :  (20039, 96, 96)
Processing folder  datasets/cropped_96_96/ 06
(23461, 2, 96, 96)  :  (23461, 96, 96)
Processing folder  datasets/cropped_96_96/ 07
(26920, 2, 96, 96)  :  (26920, 96, 96)
Processing folder  datasets/cropped_96_96/ 08
(30159, 2, 96, 96)  :  (30159, 96, 96)
Processing folder  datasets/cropped_96_96/ 09
(33077, 2, 96, 96)  :  (33077, 96, 96)
Processing folder  datasets/cropped_96_96/ 10
(35811, 2, 96, 96)  :  (35811, 96, 96)
Processing folder  datasets/cropped_96_96/ 11
(39244, 2, 96, 96)  :  (39244, 96, 96)
Processing folder  datasets/crop

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtrash, ytrain, ytrash = train_test_split(X, y, test_size=0.35, random_state=123)
Xtrain, Xtest, ytrain, ytest = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=123)

In [6]:
np.save("datasets/interpolation/X_train",Xtrain)
np.save("datasets/interpolation/y_train",ytrain)
np.save("datasets/interpolation/X_test",Xtest)
np.save("datasets/interpolation/y_test",ytest)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/interpolation/X_train.npy'

### Making dataset smaller

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split

Here goes the training part of dataset.

In [ ]:
Xtrain = np.load("datasets/dataset_interpolation_96/X_train.npy")
ytrain = np.load("datasets/dataset_interpolation_96/y_train.npy")

In [5]:
Xtrain.shape

(149872, 2, 96, 96)

In [4]:
Xtrain, Xtrash, ytrain, ytrash = train_test_split(Xtrain, ytrain, test_size=0.1, random_state=123)

In [7]:
np.save("datasets/dataset_interpolation_96/X_train",Xtrain)
np.save("datasets/dataset_interpolation_96/y_train",ytrain)

Here goes the test part of dataset.

In [2]:
Xtest = np.load("datasets/dataset_interpolation_96/X_test.npy")
ytest = np.load("datasets/dataset_interpolation_96/y_test.npy")

In [3]:
Xtest.shape

(55509, 2, 96, 96)

In [4]:
Xtest, Xtrash, ytest, ytrash = train_test_split(Xtest, ytest, test_size=0.1, random_state=123)

In [5]:
np.save("datasets/dataset_interpolation_96/X_test",Xtest)
np.save("datasets/dataset_interpolation_96/y_test",ytest)